### 1. Cài đặt và nhập thư viện

In [ ]:
%pip install selenium
%pip install selenium-stealth

In [1]:
import time
import pandas as pd

from concurrent.futures import ThreadPoolExecutor
from threading import Lock

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium_stealth import stealth

<table>
  <thead>
    <tr>
      <th>Thư viện</th>
      <th>Mô tả</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><code>selenium</code></td>
      <td>Thư viện điều khiển trình duyệt tự động, cho phép mô phỏng hành vi người dùng như click, nhập liệu, lấy dữ liệu từ trang web.</td>
    </tr>
    <tr>
      <td><code>selenium_stealth</code></td>
      <td>Ngụy trang trình duyệt Selenium để tránh bị website phát hiện là bot.</td>
    </tr>
    <tr>
      <td><code>ThreadPoolExecutor</code></td>
      <td>Chạy song song nhiều tác vụ bằng luồng (thread), tăng tốc crawl dữ liệu từ nhiều mã cổ phiếu.</td>
    </tr>
    <tr>
      <td><code>Lock</code></td>
      <td>Đảm bảo an toàn khi nhiều luồng cùng ghi file hoặc truy cập vùng dữ liệu chung.</td>
    </tr>
    <tr>
      <td><code>WebDriverWait</code>, <code>expected_conditions</code></td>
      <td>Chờ một điều kiện xảy ra như chờ phần tử xuất hiện trên trang trước khi thao tác.</td>
    </tr>
    <tr>
      <td><code>NoSuchElementException</code></td>
      <td>Xử lý lỗi khi phần tử không tồn tại, giúp chương trình tiếp tục hoạt động mà không crash.</td>
    </tr>
  </tbody>
</table>


### 2. Cấu hình Selenium

In [8]:
def setup_driver(driver_path="../../browserDrivers/chromedriver.exe"):
    chrome_options = Options()
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    #chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-infobars")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)

    service = Service(driver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)

    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )

    driver.delete_all_cookies()
    return driver


<div>
  <table>
    <thead>
      <tr>
        <th>Tham số</th>
        <th>Chức năng</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td><code>--disable-blink-features=AutomationControlled</code></td>
        <td>Ẩn dấu hiệu tự động hóa (automation) khỏi trang web. Ngăn việc phát hiện <code>navigator.webdriver = true</code>.</td>
      </tr>
      <tr>
        <td><code>--headless</code></td>
        <td>Chạy trình duyệt ở chế độ "không hiển thị giao diện". Thường dùng khi chạy server hoặc CI/CD.</td>
      </tr>
      <tr>
        <td><code>--disable-gpu</code></td>
        <td>Tắt tăng tốc phần cứng GPU, cần thiết khi chạy ở chế độ headless trên Windows.</td>
      </tr>
      <tr>
        <td><code>--disable-infobars</code></td>
        <td>Ẩn thanh thông báo "Chrome is being controlled by automated test software".</td>
      </tr>
      <tr>
        <td><code>--start-maximized</code></td>
        <td>Mở trình duyệt ở chế độ toàn màn hình (giả lập thao tác người dùng thực tế).</td>
      </tr>
      <tr>
        <td><code>--no-sandbox</code></td>
        <td>Tắt sandbox để tránh lỗi khi chạy trong container hoặc môi trường bị giới hạn quyền.</td>
      </tr>
      <tr>
        <td><code>--disable-dev-shm-usage</code></td>
        <td>Tránh sử dụng /dev/shm (bộ nhớ chia sẻ), dùng để xử lý lỗi bộ nhớ trong Docker.</td>
      </tr>
      <tr>
        <td><code>--disable-extensions</code></td>
        <td>Tắt tất cả tiện ích mở rộng của trình duyệt.</td>
      </tr>
      <tr>
        <td><code>user-agent=...</code></td>
        <td>Giả lập trình duyệt thông thường với chuỗi User-Agent tùy chỉnh.</td>
      </tr>
      <tr>
        <td><code>excludeSwitches: ["enable-automation"]</code></td>
        <td>Ngăn Chrome bật flag <code>enable-automation</code> để ẩn cảnh báo điều khiển tự động.</td>
      </tr>
      <tr>
        <td><code>useAutomationExtension: False</code></td>
        <td>Tắt extension mặc định của ChromeDriver dùng cho automation, giúp giảm dấu vết bot.</td>
      </tr>
    </tbody>
  </table>
</div>


### 3. Cấu hình tham số

In [9]:
NUM_THREADS = 3  
MAX_RETRIES = 3 
lock = Lock()

output_fields = ["date", "open", "high", "low", "close", "volume"]

symbols = ["FPT", "HPG", "VNM"]

<table>
  <thead>
    <tr>
      <th>🛠️ Biến / Hằng</th>
      <th>📝 Mô tả</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><code>NUM_THREADS</code></td>
      <td>Số luồng (threads) sẽ được sử dụng để xử lý song song. Tăng số này giúp chương trình nhanh hơn khi thu thập dữ liệu nhiều mã cổ phiếu.</td>
    </tr>
    <tr>
      <td><code>MAX_RETRIES</code></td>
      <td>Số lần thử lại tối đa nếu một thao tác (ví dụ như tải trang hoặc tìm phần tử) bị lỗi hoặc timeout.</td>
    </tr>
    <tr>
      <td><code>lock = Lock()</code></td>
      <td>Tạo một <strong>khóa luồng (thread lock)</strong> để đảm bảo rằng <strong>chỉ một luồng được thực thi đoạn mã quan trọng tại một thời điểm</strong>, giúp tránh xung đột khi nhiều luồng cùng ghi vào một tài nguyên như file hoặc biến dùng chung.</td>
    </tr>
    <tr>
      <td><code>output_fields</code></td>
      <td>Danh sách các trường dữ liệu cần lấy (ví dụ: <code>["date", "open", "high", "low", "close", "volume"]</code>). Dùng để xác định và chuẩn hóa cột khi lưu kết quả.</td>
    </tr>
    <tr>
      <td><code>symbols</code></td>
      <td>Danh sách các mã cổ phiếu cần thu thập dữ liệu, ví dụ: <code>["FPT", "HPG", "VNM"]</code>.</td>
    </tr>
  </tbody>
</table>


## 4. Định nghĩa các hàm thu thập dữ liệu

Trong phần này, chúng ta sẽ xây dựng các hàm thu thập dữ liệu chứng khoán từ website CafeF.

- `crawl_data` – Thu thập toàn bộ dữ liệu lịch sử của một mã cổ phiếu, tự động phân trang và dừng khi hết dữ liệu.
- `crawl_data_with_retry()` + `crawl_multiple_symbols()` – Tích hợp retry, đa luồng để thu thập nhiều mã cổ phiếu cùng lúc.

In [10]:
def crawl_data(driver, symbol: str):
    url = f"https://simplize.vn/co-phieu/{symbol}/lich-su-gia"
    driver.get(url)

    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "simplize-table-body")))

    all_data = []
    seen_dates = set()
    page_count = 1

    while True:
        rows = driver.find_elements(By.CSS_SELECTOR, ".simplize-table-tbody > tr")

        for row in rows:
            try:
                cols = row.find_elements(By.TAG_NAME, "td")
                if len(cols) < 8:
                    continue

                date = cols[0].text.strip()
                if date in seen_dates:
                    continue
                seen_dates.add(date)

                all_data.append({
                    "date": date,
                    "open": cols[1].text.strip(),
                    "high": cols[2].text.strip(),
                    "low": cols[3].text.strip(),
                    "close": cols[4].text.strip(),
                    "volume": cols[7].text.strip()
                })
            except:
                continue

        # Kiểm tra nút Next
        try:
            next_buttons = driver.find_elements(By.CSS_SELECTOR, ".simplize-pagination-item-link.css-11we714")
            next_btn = next_buttons[-1]

            if next_btn.get_attribute("disabled") is not None:
                print("✅ Đã đến trang cuối.")
                break

            driver.execute_script("arguments[0].click();", next_btn)
            time.sleep(2)
            page_count += 1
        except Exception as e:
            print(f"❌ Lỗi khi chuyển trang: {e}")
            break

    return pd.DataFrame(all_data)


In [11]:
def crawl_data_with_retry(symbol):
    for attempt in range(MAX_RETRIES):
        try:
            print(f"[{symbol}] Crawling (attempt {attempt + 1})...")
            driver = setup_driver()
            df = crawl_data(driver, symbol)
            driver.quit()
            return df
        except Exception as e:
            print(f"⚠️ [{symbol}] Lỗi ở lần {attempt + 1}: {e}")
            try: driver.quit()
            except: pass
            time.sleep(2)
    print(f"❌ [{symbol}] Thất bại sau {MAX_RETRIES} lần.")
    return pd.DataFrame(columns=output_fields)


In [12]:
def crawl_multiple_symbols(symbols):
    final_results = []

    with ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
        futures = {
            executor.submit(crawl_data_with_retry, symbol): symbol
            for symbol in symbols
        }

        for future in futures:
            symbol = futures[future]
            try:
                df = future.result()
                if not df.empty:
                    with lock:
                        df.to_csv(f"../../data/raw/{symbol.lower()}.csv", index=False)
                        final_results.append(df)
                        print(f"✅ [{symbol}] Đã lưu thành công ({len(df)} dòng)")
                else:
                    print(f"❌ [{symbol}] Không có dữ liệu.")
            except Exception as e:
                print(f"🔥 [{symbol}] Lỗi trong luồng: {e}")

    # Trả về 1 DataFrame tổng nếu cần
    return pd.concat(final_results, ignore_index=True) if final_results else pd.DataFrame(columns=output_fields)

In [13]:
df_all = crawl_multiple_symbols(symbols)

[FPT] Crawling (attempt 1)...
[HPG] Crawling (attempt 1)...
[VNM] Crawling (attempt 1)...
✅ Đã đến trang cuối.
✅ Đã đến trang cuối.
✅ [FPT] Đã lưu thành công (4553 dòng)
✅ [HPG] Đã lưu thành công (4350 dòng)
✅ Đã đến trang cuối.
✅ [VNM] Đã lưu thành công (4784 dòng)
